In [1]:
import os
import sys
import tensorflow as tf
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants, signature_def_utils, tag_constants, utils
from tensorflow.python.util import compat
import numpy as np
import pickle

from utils.consts import agg_ops

## training flags
tf.app.flags.DEFINE_integer('training_iteration', 101, 'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('work_dir', '/tmp', 'Working directory.')

FLAGS = tf.app.flags.FLAGS

data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './data/agg_data/%s_agg_X.pkl'
y_file = './data/agg_data/%s_agg_Y.pkl'
idx_file = './data/op_data/%s_op_idx.pkl'

def get_data(dt):
    with open(y_file%dt, 'rb') as fy:
        _Y = pickle.load(fy)

    with open(x_file%dt, 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

dev_X, dev_Y = get_data('dev')
out_classes = np.array(agg_ops)

def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(dev_X):
        return (dev_X[lw:hs][:], dev_Y[lw:hs][:])
    else:
        return (dev_X[lw:][:], dev_Y[lw:][:])

# def train():
n_input = 1500
n_hidden1 = 1024
n_hidden2 = 512
n_hidden3 = 64
n_output = 6

learning_rate = 1e-4
n_iteration = 11
batch_size = 128
dropout = 0.1

def main(_):
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_output])
    # keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

    weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
    }

    biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
    }

    layer_1 = tf.nn.tanh(tf.add(tf.matmul(X, weights['w1']), biases['b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
    # layer_drop = tf.nn.dropout(layer_3, keep_prob)
    output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(layer_3, weights['out']), biases['out']))

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    n_batches = len(dev_X)//batch_size

    # train on mini batches
    for i in range(n_iteration):
        for j in range(n_batches+1):
            (batch_x, batch_y) = get_batch(j, batch_size)
            sess.run(train_step, feed_dict={X: batch_x, Y: batch_y})

            # print loss and accuracy (per minibatch)
        if i % 5 == 0:
            minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

    ## save model
    export_path_base = "/root/projects/description2sql/notebooks/Models/models/agg"
    export_path = os.path.join(
        compat.as_bytes(export_path_base),
        compat.as_bytes(str(FLAGS.model_version))
    )
    print("Exporting model to : ", export_path_base)
    
    builder = saved_model_builder.SavedModelBuilder(export_path)
    
    classification_inputs = utils.build_tensor_info(tf.convert_to_tensor(dev_X))
    classification_outputs_classes = utils.build_tensor_info(tf.convert_to_tensor(out_classes))
    classification_outputs_scores = utils.build_tensor_info(tf.convert_to_tensor(dev_Y))
    
    classification_signature = signature_def_utils.build_signature_def(
        inputs={signature_constants.CLASSIFY_INPUTS: classification_inputs},
        outputs={signature_constants.CLASSIFY_OUTPUT_CLASSES: classification_outputs_classes,
                 signature_constants.CLASSIFY_OUTPUT_SCORES: classification_outputs_scores},
        method_name=signature_constants.CLASSIFY_METHOD_NAME
    )
    
    tensor_info_x = utils.build_tensor_info(X)
    tensor_info_y = utils.build_tensor_info(Y)
    
    prediction_signature = signature_def_utils.build_signature_def(
        inputs={'images': tensor_info_x},
        outputs={'scores': tensor_info_y},
        method_name=signature_constants.PREDICT_METHOD_NAME)
    
    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    
    builder.add_meta_graph_and_variables(
        sess, [tag_constants.SERVING],
        signature_def_map={
            'predict agg function':
                prediction_signature,
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                classification_signature,
        },
        legacy_init_op=legacy_init_op)
    
    builder.save()
    print("done exporting")
    
    
if __name__ == '__main__':
    tf.app.run()

#     test_X, test_Y = get_data('test')
#     test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
#     print("\nAccuracy on test set : ", test_accuracy)

Iteration 0 	| Batch 19 	| Loss = 1.4062091 	| Accuracy = 0.5205479
Iteration 5 	| Batch 19 	| Loss = 0.38727227 	| Accuracy = 0.89041096
Iteration 10 	| Batch 19 	| Loss = 0.2682096 	| Accuracy = 0.9315069
Exporting model to :  /root/projects/description2sql/notebooks/Models/models/agg
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /root/projects/description2sql/notebooks/Models/models/agg/1/saved_model.pb
done exporting


SystemExit: 

/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 

In [7]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from pprint import pprint
GRAPH_PB_PATH = '/root/projects/description2sql/notebooks/Models/models/agg/1/saved_model.pb'
with tf.Session() as sess:
    print("load graph")
    with gfile.FastGFile(GRAPH_PB_PATH, 'rb') as f:
        graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes = [n for n in graph_def.node]
    names = [t.name for t in graph_nodes]
    pprint(names)

load graph


DecodeError: Error parsing message

In [3]:
!ls /root/projects/description2sql/notebooks/Models/models/agg/1

saved_model.pb	variables


In [5]:
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

In [6]:
export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

SyntaxError: invalid syntax (<ipython-input-6-436d41b4ad91>, line 1)